In [1]:
!pip3 install boto3

In [2]:
import boto3
import json

def detect_faces(photo):

    client=boto3.client('rekognition')
    
    with open(photo, 'rb') as image:
        response = client.detect_faces(Image={'Bytes': image.read()}, Attributes=['ALL'])
    
  #  response = client.detect_faces(Image={'S3Object':{'Bucket':bucket,'Name':photo}},Attributes=['ALL'])

    print('Detected faces for ' + photo)    
    for faceDetail in response['FaceDetails']:
        print('The detected face is between ' + str(faceDetail['AgeRange']['Low']) 
              + ' and ' + str(faceDetail['AgeRange']['High']) + ' years old')
       
        
        
   #     print('Here are the other attributes:')
   #     print(json.dumps(faceDetail, indent=4, sort_keys=True))

		# Access predictions for individual face details and print them
        print("Gender: " + str(faceDetail['Gender']))
        print("Smile: " + str(faceDetail['Smile']))
        print("Eyeglasses: " + str(faceDetail['Eyeglasses']))
        print("Emotions: " + str(faceDetail['Emotions'][0]))
        
        age = faceDetail['AgeRange']['High'] + faceDetail['AgeRange']['Low']/2
        gender = faceDetail['Gender']['Value']
        emotion = faceDetail['Emotions'][0]['Type']
        
        result = age, gender, emotion

    return result
def main():
    photo='sungsu2.jpg'
 #   face_count=detect_faces(photo)
 #   print("Faces detected: " + str(face_count))
    print('메인실행됨')


if __name__ == "__main__":
    main()

메인실행됨


In [3]:
#Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3

def compare_faces(sourceFile, targetFile):

    client=boto3.client('rekognition')
   
    imageSource=open(sourceFile,'rb')
    imageTarget=open(targetFile,'rb')

    response=client.compare_faces(SimilarityThreshold=0,
                                  SourceImage={'Bytes': imageSource.read()},
                                  TargetImage={'Bytes': imageTarget.read()})
    
    for faceMatch in response['FaceMatches']:
        position = faceMatch['Face']['BoundingBox']
        similarity = str(faceMatch['Similarity'])
#         print('The face at ' +
#                str(position['Left']) + ' ' +
#                str(position['Top']) +
#                ' matches with ' + similarity + '% confidence')

    imageSource.close()
    imageTarget.close()     
    return similarity       

def main():
    source_file='source.jpg'
    target_file='target.jpg'
#     face_matches=compare_faces(source_file, target_file)
#     print("Face matches: " + str(face_matches))
    print('메인 실행 됨')


if __name__ == "__main__":
    main()




메인 실행 됨


In [4]:
#Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3
import json

def recognize_celebrities(photo):

    
    client=boto3.client('rekognition')

    with open(photo, 'rb') as image:
        response = client.recognize_celebrities(Image={'Bytes': image.read()})

    print('Detected faces for ' + photo)    
    for celebrity in response['CelebrityFaces']:
#         print ('Name: ' + celebrity['Name'])
#         print ('Id: ' + celebrity['Id'])
#         print ('KnownGender: ' + celebrity['KnownGender'])
#         print ('Smile: ' + celebrity['Smile'])
#         print ('Position:')
#         print ('   Left: ' + '{:.2f}'.format(celebrity['Face']['BoundingBox']['Height']))
#         print ('   Top: ' + '{:.2f}'.format(celebrity['Face']['BoundingBox']['Top']))
#         print ('Info')
#         for url in celebrity['Urls']:
#             print ('   ' + url)
#         print
        name = celebrity['Name']
    if len(response['CelebrityFaces']) == 0:
        name = '아이유'
  
    return name

def main():
#     photo='moviestars.jpg'

#     celeb_count=recognize_celebrities(photo)
#     print("Celebrities detected: " + str(celeb_count))
    print('메인 실행 됨')


if __name__ == "__main__":
    main()

메인 실행 됨


In [ ]:
from flask import Flask, render_template, request, redirect
from werkzeug.utils import secure_filename

app = Flask(__name__)

@app.route('/')
def hello():
    return render_template('home.html')

@app.route('/add', methods=['GET', 'POST'])
def add_number():
    
    if request.method == 'GET':
        num1 = request.args['num1']
        num2 = request.args['num2']
        result = int(num1) + int(num2)
    else:
        num1 = request.form['num1']
        num2 = request.form['num2']
        result = int(num1) + int(num2)
        
    return '두 수의 합은 {}입니다'.format(result)

@app.route('/move', methods=['GET', 'POST'])
def move():
    
    if request.method == 'GET':
        url = 'https://'
        url += request.args['url']
        # 만약, url이 https://www.naver.com
        return redirect(url)
    
    return '주소 없음'
    
@app.route('/detect', methods=['GET', 'POST'])
def detect():
    
    if request.method == 'POST':
        f = request.files['file']
        f.save(secure_filename(f.filename))
        
        photo = f.filename
        age, gender, emotion = detect_faces(photo)
       
        
    return '나이 : {}세 / 성별 : {} / 가장 높은 감정 : {}'.format(age,gender,emotion)
    
@app.route('/compare', methods=['GET', 'POST'])
def compare():
    if request.method == 'POST':
        f = request.files['file']
        f.save(secure_filename(f.filename))
        
        source = f.filename
        target = 'target.jpg'
        similarity = compare_faces(source, target)
        
    return '본인일 확률은 {}%입니다.'.format(similarity)

@app.route('/celeb',methods=['GET', 'POST'] )
def celeb():
    
     if request.method == 'POST':
        f = request.files['file']
        f.save(secure_filename(f.filename))
        
        photo = f.filename
        name = recognize_celebrities(photo)
     return '닮은 연예인은 {}입니다.'.format(name)
    
if __name__ == '__main__':
    app.run('172.30.1.58',5028)
    
    

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://172.30.1.58:5028/ (Press CTRL+C to quit)


In [ ]:
import calculator

print(calculator.add(5,3))
print(calculator.sun(5,3))
print(calculator.mul(5,3))
print(calculator.div(5,3))